<a href="https://colab.research.google.com/github/ROARMarketingConcepts/Data-Analysis-Projects/blob/master/Home_Credit_Default_Risk_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Home Credit Default Risk Analysis

Many people struggle to get loans due to insufficient or non-existent credit histories. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

**Home Credit Group**

Home Credit strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

### Analysis performed by:

Ken Wood

Marketing Data Scientist

ken@roarmarketingconcepts.com




---



### Mount the Google Drive where the dataset files are located...

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# !apt install proj-bin libproj-dev libgeos-dev

In [0]:
# !pip install https://github.com/matplotlib/basemap/archive/v1.1.0.tar.gz
# !sudo pip install -U git+https://github.com/matplotlib/basemap.git

### Install some necessary packages to perform the required analysis...

In [0]:
# !pip install -U scikit-learn
# !pip install --user --upgrade tables

# !pip install sqlalchemy
# !pip install psycopg2-binary

import pandas as pd
import numpy as np
import sklearn
import scipy

import matplotlib.pyplot as plt
from matplotlib import interactive
plt.rc("font", size=14)
from pylab import scatter, show, legend, xlabel, ylabel

# from mpl_toolkits.basemap import Basemap

import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [0]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Let's list the datasets available to us...

In [0]:
import os

print('### Home Credit Default Risk Analysis ###\n')
for idx, file in enumerate(os.listdir('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files')):
    print(idx, '-', file)

### Home Credit Default Risk Analysis ###

0 - HomeCredit_columns_description.csv
1 - sample_submission.csv
2 - application_test.csv
3 - bureau_balance.csv
4 - bureau.csv
5 - credit_card_balance.csv
6 - installments_payments.csv
7 - POS_CASH_balance.csv
8 - previous_application.csv
9 - home_credit.png
10 - HomeCredit_columns_description.gsheet
11 - application_train_files


### Load the datasets...

In [0]:
# application_train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/application_train_files/application_train.csv')
bureau_balance = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/bureau_balance.csv')
bureau = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/bureau.csv')
# credit_card_balance = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/credit_card_balance.csv')
# installments_payments = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/installments_payments.csv')
# POS_CASH_balance = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/POS_CASH_balance.csv')
# previous_application = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/previous_application.csv')

### Let's get some preliminary information about each of the datasets...

In [0]:
application_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [0]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### Separate 'application_train' into the numerical and categorical variables...

In [0]:
application_train_index_target = application_train[['SK_ID_CURR','TARGET']]
application_train_cat = application_train.select_dtypes(include=['object'])
application_train_num =  application_train.select_dtypes(include=['int64','float64'])
application_train_num = application_train_num.drop(['SK_ID_CURR','TARGET'],axis=1)

### Let’s convert the categorical variables from text to numbers. First, we take care of the missing values using pandas 'ffill' method (propagate last valid observation forward to next valid).  Then we use the sklearn function **OrdinalEncoder** which encodes categorical features as an integer array.

In [0]:
application_train_cat.fillna(method='ffill',inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [0]:
from sklearn.preprocessing import OrdinalEncoder

In [0]:
ordinal_encoder = OrdinalEncoder()
application_train_cat_encoded = ordinal_encoder.fit_transform(application_train_cat)
application_train_cat_encoded[:10]

array([[ 0.,  1.,  0.,  1.,  6.,  7.,  4.,  3.,  1.,  8.,  6.,  5.,  2.,
         0.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  4.,  1.,  1.,  1.,  3.,  1., 39.,  2.,
         0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  6.,  7.,  4.,  3.,  1.,  8.,  1., 11.,  2.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  6.,  7.,  4.,  0.,  1.,  8.,  6.,  5.,  2.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  6.,  7.,  4.,  3.,  1.,  3.,  4., 37.,  2.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  5.,  4.,  4.,  1.,  1.,  8.,  6., 33.,  2.,
         0.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  6.,  1.,  1.,  1.,  1.,  0.,  3.,  5.,  2.,
         0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  6.,  4.,  1.,  1.,  1., 10.,  1., 33.,  2.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  3.,  4.,  1.,  1., 10.,  6., 57.,  2.,
         0.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  6.,  7.,  4.,  3.,  1.,  8.,  4.,  9.,  2.,
         0.,  0.,  0.]])

In [0]:
ordinal_encoder.categories_

[array(['Cash loans', 'Revolving loans'], dtype=object),
 array(['F', 'M', 'XNA'], dtype=object),
 array(['N', 'Y'], dtype=object),
 array(['N', 'Y'], dtype=object),
 array(['Children', 'Family', 'Group of people', 'Other_A', 'Other_B',
        'Spouse, partner', 'Unaccompanied'], dtype=object),
 array(['Businessman', 'Commercial associate', 'Maternity leave',
        'Pensioner', 'State servant', 'Student', 'Unemployed', 'Working'],
       dtype=object),
 array(['Academic degree', 'Higher education', 'Incomplete higher',
        'Lower secondary', 'Secondary / secondary special'], dtype=object),
 array(['Civil marriage', 'Married', 'Separated', 'Single / not married',
        'Unknown', 'Widow'], dtype=object),
 array(['Co-op apartment', 'House / apartment', 'Municipal apartment',
        'Office apartment', 'Rented apartment', 'With parents'],
       dtype=object),
 array(['Accountants', 'Cleaning staff', 'Cooking staff', 'Core staff',
        'Drivers', 'HR staff', 'High skill tech 

### If we want to put it back into a Pandas DataFrame, it’s simple:

In [0]:
application_train_cat = pd.DataFrame(application_train_cat_encoded,columns = application_train_cat.columns)
application_train_cat.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,0.0,1.0,0.0,1.0,6.0,7.0,4.0,3.0,1.0,8.0,6.0,5.0,2.0,0.0,5.0,0.0
1,0.0,0.0,0.0,0.0,1.0,4.0,1.0,1.0,1.0,3.0,1.0,39.0,2.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,6.0,7.0,4.0,3.0,1.0,8.0,1.0,11.0,2.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,6.0,7.0,4.0,0.0,1.0,8.0,6.0,5.0,2.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,6.0,7.0,4.0,3.0,1.0,3.0,4.0,37.0,2.0,0.0,0.0,0.0


### Now we need to deal with missing numerical values in 'application_train'. Scikit-Learn provides a handy class to take care of missing values: **Imputer**. First, we need to create an Imputer instance, specifying that we want to replace each attribute’s missing values with the median of that attribute:

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler())
        ])

In [0]:
application_train_num_encoded = num_pipeline.fit_transform(application_train_num)
application_train_num_encoded

array([[-0.57753784,  0.14212925, -0.47809496, ..., -0.26994654,
        -0.30861959, -0.44092567],
       [-0.57753784,  0.42679193,  1.7254498 , ..., -0.26994654,
        -0.30861959, -1.00733095],
       [-0.57753784, -0.4271961 , -1.15288792, ..., -0.26994654,
        -0.30861959, -1.00733095],
       ...,
       [-0.57753784, -0.06662338,  0.19537871, ...,  0.89717516,
        -0.30861959, -0.44092567],
       [-0.57753784,  0.00928667, -0.56875681, ..., -0.26994654,
        -0.30861959, -1.00733095],
       [-0.57753784, -0.04764587,  0.18875991, ...,  2.06429685,
        -0.30861959, -0.44092567]])

In [0]:
application_train_num = pd.DataFrame(application_train_num_encoded,columns = application_train_num.columns)
application_train_num = pd.concat([application_train_index_target,application_train_num],axis=1)
application_train_num.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,-0.577538,0.142129,-0.478095,-0.166143,-0.507236,-0.149452,1.506880,-0.456215,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
1,100003,0,-0.577538,0.426792,1.725450,0.592683,1.600873,-1.252750,-0.166821,-0.460115,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331
2,100004,0,-0.577538,-0.427196,-1.152888,-1.404669,-1.092145,-0.783451,-0.689509,-0.453299,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331
3,100006,0,-0.577538,-0.142533,-0.711430,0.177874,-0.653463,-0.928991,-0.680114,-0.473217,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
4,100007,0,-0.577538,-0.199466,-0.213734,-0.361749,-0.068554,0.563570,-0.892535,-0.473210,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331


In [0]:
application_train_coded = pd.concat([application_train_num,application_train_cat],axis=1)
# application_train_coded.to_csv(r'/gdrive/My Drive/Colab Notebooks/Home Credit Default Risk/files/application_train_files/application_train_coded.csv')
application_train_coded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to EMERGENCYSTATE_MODE
dtypes: float64(120), int64(2)
memory usage: 286.2 MB


In [0]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
SK_ID_CURR                int64
SK_ID_BUREAU              int64
CREDIT_ACTIVE             object
CREDIT_CURRENCY           object
DAYS_CREDIT               int64
CREDIT_DAY_OVERDUE        int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG        int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE               object
DAYS_CREDIT_UPDATE        int64
AMT_ANNUITY               float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [0]:
bureau.isnull().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64

In [0]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [0]:
bureau_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
SK_ID_BUREAU      int64
MONTHS_BALANCE    int64
STATUS            object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [0]:
bureau_balance.isnull().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [0]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [0]:
bureau = bureau.merge(bureau_balance, how='left', on='SK_ID_BUREAU')

In [0]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25121815 entries, 0 to 25121814
Data columns (total 19 columns):
SK_ID_CURR                int64
SK_ID_BUREAU              int64
CREDIT_ACTIVE             object
CREDIT_CURRENCY           object
DAYS_CREDIT               int64
CREDIT_DAY_OVERDUE        int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG        int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE               object
DAYS_CREDIT_UPDATE        int64
AMT_ANNUITY               float64
MONTHS_BALANCE            float64
STATUS                    object
dtypes: float64(9), int64(6), object(4)
memory usage: 3.7+ GB


In [0]:
bureau.isnull().sum()

SK_ID_CURR                       0
SK_ID_BUREAU                     0
CREDIT_ACTIVE                    0
CREDIT_CURRENCY                  0
DAYS_CREDIT                      0
CREDIT_DAY_OVERDUE               0
DAYS_CREDIT_ENDDATE        1232569
DAYS_ENDDATE_FACT          5978831
AMT_CREDIT_MAX_OVERDUE    18130741
CNT_CREDIT_PROLONG               0
AMT_CREDIT_SUM                  13
AMT_CREDIT_SUM_DEBT        4234392
AMT_CREDIT_SUM_LIMIT      10671361
AMT_CREDIT_SUM_OVERDUE           0
CREDIT_TYPE                      0
DAYS_CREDIT_UPDATE               0
AMT_ANNUITY               10495530
MONTHS_BALANCE              942074
STATUS                      942074
dtype: int64

In [0]:
grouped = bureau.groupby(['SK_ID_CURR','SK_ID_BUREAU'])['SK_ID_BUREAU'].count()

In [27]:
grouped

SK_ID_CURR  SK_ID_BUREAU
100001      5896630         29
            5896631         30
            5896632         29
            5896633         52
            5896634         19
            5896635          2
            5896636         11
100002      6113835         16
            6158903         16
            6158904         22
            6158905         16
            6158906         16
            6158907         16
            6158908          4
            6158909          4
100003      5885877          1
            5885878          1
            5885879          1
            5885880          1
100004      6829133          1
            6829134          1
100005      6735200         13
            6735201          5
            6735202          3
100007      5987200          1
100008      6491432          1
            6491433          1
            6491434          1
100009      5650996          1
            5650997          1
                            ..
456249      53

In [0]:
credit_card_balance.info()

In [0]:
credit_card_balance.isnull().sum()

In [0]:
credit_card_balance.head()

In [0]:
installments_payments.info()

In [0]:
installments_payments.isnull().sum()

In [0]:
installments_payments.head()

In [0]:
POS_CASH_balance.info()

In [0]:
POS_CASH_balance.isnull().sum()

In [0]:
POS_CASH_balance.head()

In [0]:
previous_application.info()

In [0]:
previous_application.isnull().sum()

In [0]:
previous_application.head()

In [0]:
application_train = application_train.merge(bureau,how='left',on='SK_ID_CURR')
application_train = application_train.merge(bureau_balance,how='left',on='SK_ID_BUREAU')

application_train = application_train.merge(previous_application,how='left',on='SK_ID_CURR')
application_train = application_train.merge(POS_CASH_balance,how='left',on='SK_ID_PREV')
application_train = application_train.merge(installments_payments,how='left',on='SK_ID_PREV')
application_train = application_train.merge(credit_card_balance,how='left',on='SK_ID_PREV')

### Now, we want to join information from the other datasets into the marketing funnel. Based on the schema diagram, 'order_items' should be the next dataset we merge.  Let's get some information about the 'order_items' dataset... We will merge it with the marketing funnel on the 'seller_id' column.

In [0]:
order_items.info()

In [0]:
mf_items = mf.merge(order_items, how='left', on='seller_id')
mf_items['mql_id'].nunique()

In [0]:
mf_items.info()

### Let's take a look at the 'sellers' dataset...

In [0]:
sellers.info()

In [0]:
sellers['seller_id'].nunique()

### So we can conclude that the dataset has 3095 unique sellers.  Let's merge the seller information for each of the leads in the marketing funnel.

In [0]:
mf_sellers = mf.merge(sellers, how='left', on='seller_id')
mf_sellers['mql_id'].nunique()

In [0]:
mf_sellers['seller_id'].nunique()

In [0]:
geolocation.info()

###Now, in order to merge the 'geolocation' information into the marketing funnel on *zip_code_prefix*, we need to remove the duplicate values in 'geolocation' and then change the column names in both datasets.

In [0]:
geolocation['geolocation_zip_code_prefix'].value_counts()

In [0]:
geolocation = geolocation.drop_duplicates('geolocation_zip_code_prefix',keep='first')
geolocation['geolocation_zip_code_prefix'].value_counts()

In [0]:
mf_sellers.rename(columns = {'seller_zip_code_prefix':'zip_code_prefix'},inplace=True)
geolocation.rename(columns = {'geolocation_zip_code_prefix':'zip_code_prefix'},inplace=True)

### We verify that the correct columns have been renamed...

In [0]:
 # mf_sellers.info()

In [0]:
# geolocation.info()

In [0]:
mf_sellers = pd.merge(mf_sellers,geolocation[['zip_code_prefix','geolocation_lat','geolocation_lng']], how = 'left',on='zip_code_prefix')
mf_sellers.info()

In [0]:
mf_sellers.head()

### Let's take a look at the frequency of the unique values in each column of the 2 marketing funnels. For now, we will skip the id and date columns to keep our analysis clean.

## mf_items:

<class 'pandas.core.frame.DataFrame'>


### Data columns (total 23 columns):

mql_id                           12664 non-null object

first_contact_date               12664 non-null datetime64[ns]

landing_page_id                  12664 non-null object

origin                           12597 non-null object

seller_id                        5506 non-null object

sdr_id                           5506 non-null object

sr_id                            5506 non-null object

won_date                         5506 non-null datetime64[ns]

business_segment                 5505 non-null object

lead_type                        5455 non-null object

lead_behaviour_profile           3679 non-null object

has_company                      88 non-null object

has_gtin                         89 non-null object

average_stock                    91 non-null object

business_type                    5494 non-null object

declared_product_catalog_size    69 non-null float64

declared_monthly_revenue         5506 non-null float64

order_id                         5044 non-null object

order_item_id                    5044 non-null float64

product_id                       5044 non-null object

shipping_limit_date              5044 non-null datetime64[ns]

price                            5044 non-null float64

freight_value                    5044 non-null float64

dtypes: datetime64[ns](3), float64(5), object(15)

memory usage: 2.3+ MB

In [0]:
columns = ['origin','business_segment','lead_type','lead_behaviour_profile','has_company','has_gtin','average_stock',
           'business_type']

for col in columns:
    print(mf_items[col].value_counts(),'\n\n',mf_items[col].value_counts(normalize=True),'\n\n')
    

In [0]:
mf_items_origin = mf_items['origin'].value_counts(normalize=True).reset_index()

In [0]:
fig,ax = plt.subplots(figsize=(14,6))

order = []                        # Set up order for barchart
for row in mf_items_origin['index']:
  order.append(row)

sns.countplot(mf_items['origin'].sort_values(), order=order, palette="Blues_d")
ax.set_xlabel('Lead Source')
ax.set_title('Lead Source Distribution')

plt.show()

In [0]:
fig,ax = plt.subplots(figsize=(14,6))

order = []                        # Set up order for barchart
for row in mf_items_origin['index']:
  order.append(row)

sns.barplot(x ='index',y = 'origin', data = mf_items_origin, order=order, palette="Blues_d")
ax.set_xlabel('Lead Source')
ax.set_ylabel('Normalized Count - % of Total')
ax.set_title('Lead Source Distribution')

plt.show()

In [0]:
mf_items_business_segment = mf_items['business_segment'].value_counts(normalize=True).reset_index()

In [0]:
fig,ax = plt.subplots(figsize=(14,6))

order = []                        # Set up order for barchart
for row in mf_items_business_segment['index']:
  order.append(row)

graph = sns.barplot(x ='index',y = 'business_segment', data = mf_items_business_segment, order=order, palette="Blues_d")
ax.set_xlabel('Business Segment')
ax.set_ylabel('Normalized Count - % of Total')
ax.set_title('Business Segment Distribution')
graph.set_xticklabels(order,rotation=90)

plt.show()

In [0]:
mf_items_lead_type = mf_items['lead_type'].value_counts(normalize=True).reset_index()

In [0]:
fig,ax = plt.subplots(figsize=(14,6))

order = []                        # Set up order for barchart
for row in mf_items_lead_type['index']:
  order.append(row)

graph = sns.barplot(x ='index',y ='lead_type', data = mf_items_lead_type, order=order, palette="Blues_d")
ax.set_xlabel('Lead Type')
ax.set_ylabel('Normalized Count - % of Total')
ax.set_title('Lead Type Distribution')
# graph.set_xticklabels(order,rotation=90)

plt.show()

In [0]:
mf_items_lead_behaviour_profile = mf_items['lead_behaviour_profile'].value_counts(normalize=True).reset_index()

In [0]:
fig,ax = plt.subplots(figsize=(14,6))

order = []                        # Set up order for barchart
for row in mf_items_lead_behaviour_profile['index']:
  order.append(row)

graph = sns.barplot(x ='index',y ='lead_behaviour_profile', data = mf_items_lead_behaviour_profile, order=order, palette="Blues_d")
ax.set_xlabel('Lead Behaviour Profile')
ax.set_ylabel('Normalized Count - % of Total')
ax.set_title('Lead Behaviour Profile Distribution')
# graph.set_xticklabels(order,rotation=90)

plt.show()

In [0]:
mf_items_business_type = mf_items['business_type'].value_counts(normalize=True).reset_index()

In [0]:
fig,ax = plt.subplots(figsize=(4,4))

order = []                        # Set up order for barchart
for row in mf_items_business_type['index']:
  order.append(row)

graph = sns.barplot(x ='index',y ='business_type', data = mf_items_business_type, order=order, palette="Blues_d")
ax.set_xlabel('Business Type Profile')
ax.set_ylabel('Normalized Count - % of Total')
ax.set_title('Business Type Profile Distribution')
# graph.set_xticklabels(order,rotation=90)

plt.show()

### Now, let's look at the 'mf_sellers' dataset.  We would like to get more information about where the sellers are located.  We can use the 'geolocation_lat' and 'geolocation_lng' fields and Python's 'Basemap' package to plot geographies.  We start with a basic Basemap plot of Earth.

In [0]:
map = Basemap()

map.drawcoastlines()

plt.show()
plt.savefig('test.png')

In [0]:
map = Basemap(projection='ortho', 
              lat_0=0, lon_0=-90)

#Fill the globe with a blue color 
map.drawmapboundary(fill_color='aqua')

#Fill the continents with the land color
map.fillcontinents(color='coral',lake_color='aqua')

map.drawcoastlines()

plt.show()

In [0]:
mf_sellers.info()

### Determine range to print based on min, max lat and long of the data

In [0]:
lat = np.asarray(mf_sellers['geolocation_lat'].dropna())
lng = np.asarray(mf_sellers['geolocation_lng'].dropna())

In [0]:
from geopy.geocoders import Nominatim

geolocator = Nominatim()

def PlotCity(city):
  loc1 = geolocator.geocode(city)
  x, y = map(loc1.longitude, loc1.latitude)
  ax.plot(x, y, 'ok', markersize=10)
  plt.text(x, y, city, fontsize=12,fontweight='bold')
  return
 

In [0]:
margin = .2                     # buffer to add to the range
lat_min = min(lat) - margin
lat_max = max(lat) + margin
lng_min = min(lng) - margin
lng_max = max(lng) + margin

# create map using BASEMAP
fig,ax = plt.subplots(figsize=(14,8))

map = Basemap(llcrnrlon=lng_min,
            llcrnrlat=lat_min,
            urcrnrlon=lng_max,
            urcrnrlat=lat_max,
            lat_0=(lat_max - lat_min)/2,
            lon_0=(lng_max-lng_min)/2,
            projection='merc',
            resolution = 'h',
            area_thresh=10000.,
            )
map.drawcoastlines()
map.drawcountries()
map.drawstates()
map.drawmapboundary(fill_color='#46bcec')
map.fillcontinents(color = 'white',lake_color='#46bcec')

# convert lat and long to map projection coordinates
x,y = map(lng, lat)

# plot points as red dots

map.scatter(x,y, marker = 'o', s=20,color='r',alpha=0.35, zorder=10)

# Find and plot major cities

city_list = ['Sao Paulo','Rio de Janeiro','Curitiba','Brasilia','Natal','Uberaba','Salvador-BA','Porto Alegre','Itajai']

for city in city_list:
  PlotCity(city)

plt.title("Olist Seller Locations in Brazil",fontsize=20,fontweight='bold')
plt.show()

### Let's look at characteristics of the closed deals...

In [0]:
cd.info()

### If we merge 'cd' with 'order_items' we find out about sellers and items that associated with closed deals.

In [0]:
cd_items = cd.merge(order_items,how='left',on='seller_id')
print(cd_items['mql_id'].nunique())
cd_items.info()

### Let's determine how many 'order_id's are associated with each closed deal 'mql_id'.  We'll also determine the revenue associated with each if these 'mql_id's.

In [0]:
order_ids_per_mql = cd_items.groupby(['mql_id'])['order_id'].count().reset_index()
order_ids_per_mql = order_ids_per_mql[order_ids_per_mql.order_id != 0]  # Delete rows where count(order_id) = 0
order_ids_per_mql.rename(columns = {'order_id':'num_order_ids'},inplace=True)
order_ids_per_mql.sort_values(by='num_order_ids',ascending=False)

### Now we can merge the 'orders' and 'order_payments' and 'order_reviews' information for each 'order_id' in 'cd_items'. However, before we do that, let's add a column to the 'order_payments' dataset.

In [0]:
order_payments.head()

### We will add a column called 'total_payment' which is the product of the 'payment_installments' and 'payment_value'.

In [0]:
order_payments['total_payment'] = order_payments['payment_installments']*order_payments['payment_value']

In [0]:
from functools import reduce

dfs_to_merge = [cd_items,orders,order_payments,order_reviews]
cd_order_info = reduce(lambda left,right: pd.merge(left,right,on=['order_id'],how='left'), dfs_to_merge)
cd_order_info.info()

### What is the revenue value for each closed deal 'mql_id'?

In [0]:
revenue_per_mql = cd_order_info.groupby(['mql_id'])['total_payment'].sum().reset_index()
revenue_per_mql = revenue_per_mql[revenue_per_mql.total_payment != 0]  # Delete rows where sum(total_payment) = 0
top_20_mqls= revenue_per_mql.sort_values(by='total_payment',ascending=False).head(20)
top_20_mqls

In [0]:
import matplotlib.ticker as mtick

fig,ax = plt.subplots(figsize=(14,6))

order = []                        # Set up order for barchart
for row in top_20_mqls['mql_id']:
  order.append(row)

graph = sns.barplot(x ='mql_id',y ='total_payment', data = top_20_mqls, order=order, palette="Blues_d")

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick) 
ax.set_xlabel('Closed Deal mql_id')
ax.set_ylabel('Revenue')
ax.set_title('Top 20 Closed Deal mql_ids by Revenue',fontsize=20,fontweight='bold')
graph.set_xticklabels(order,rotation=90)

plt.show()

### Let's merge the customer information into the 'cd_items' dataset...

In [0]:
cd_order_customer_info = cd_order_info.merge(customers,how='left', on='customer_id')
cd_order_customer_info.info()

### Let's merge the customer geographical location info into the 'cd_order_customer_info' dataset.

In [0]:
cd_order_customer_info.rename(columns = {'customer_zip_code_prefix':'zip_code_prefix'},inplace=True)
cd_order_customer_info = pd.merge(cd_order_customer_info,geolocation[['zip_code_prefix','geolocation_lat','geolocation_lng']], how = 'left',on='zip_code_prefix')
cd_order_customer_info.info()

In [0]:
num_customers_per_cd_mql = cd_order_customer_info.groupby(['mql_id'])['customer_id'].count().reset_index()
num_customers_per_cd_mql = num_customers_per_cd_mql[num_customers_per_cd_mql.customer_id !=0]
num_customers_per_cd_mql.rename(columns = {'customer_id': 'num_customer_ids'},inplace=True)
num_customers_per_cd_mql

In [0]:
num_products_per_cd_mql = cd_order_customer_info.groupby(['mql_id'])['product_id'].count().reset_index()
num_products_per_cd_mql = num_products_per_cd_mql[num_products_per_cd_mql.product_id !=0]
num_products_per_cd_mql.rename(columns = {'product_id': 'num_product_ids'},inplace=True)
num_products_per_cd_mql

### Let's plot the customer locations for the closed deal 'mql_id's...

In [0]:
lat = np.asarray(cd_order_customer_info['geolocation_lat'].dropna())
lng = np.asarray(cd_order_customer_info['geolocation_lng'].dropna())

In [0]:
margin = .2                     # buffer to add to the range
lat_min = min(lat) - margin
lat_max = max(lat) + margin
lng_min = min(lng) - margin
lng_max = max(lng) + margin

# create map using BASEMAP
fig,ax = plt.subplots(figsize=(14,8))

map = Basemap(llcrnrlon=lng_min,
            llcrnrlat=lat_min,
            urcrnrlon=lng_max,
            urcrnrlat=lat_max,
            lat_0=(lat_max - lat_min)/2,
            lon_0=(lng_max-lng_min)/2,
            projection='merc',
            resolution = 'h',
            area_thresh=10000.,
            )
map.drawcoastlines()
map.drawcountries()
map.drawstates()
map.drawmapboundary(fill_color='#46bcec')
map.fillcontinents(color = 'white',lake_color='#46bcec')

# convert lat and long to map projection coordinates
x,y = map(lng, lat)

# plot points as red dots

map.scatter(x,y, marker = 'o',s = 10, color='r',alpha=0.35, zorder=10)

# Find and plot major cities

city_list = ['Sao Paulo','Rio de Janeiro','Curitiba','Brasilia','Natal','Uberaba','Salvador-BA','Porto Alegre','Itajai']

for city in city_list:
  PlotCity(city)

plt.title("Olist Customer Locations in Brazil",fontsize=20,fontweight='bold')
plt.show()

### Let's get the first contact date for each of the closed deal mql_id's.  We can get that from the 'mql' dataset.

In [0]:
cd_order_customer_info = cd_order_info.merge(mql[['mql_id','first_contact_date']],how='inner', on='mql_id')
cd_order_customer_info.info()

In [0]:
cd_order_customer_info['lead_conversion_time'] = cd_order_customer_info['won_date'] - cd_order_customer_info['first_contact_date']
cd_order_customer_info['lead_conversion_time'] = cd_order_customer_info['lead_conversion_time'].dt.days  # Strip out days from timedelta variable
cd_order_customer_info['lead_conversion_time'] = cd_order_customer_info['lead_conversion_time'].apply(np.float32)
cd_order_customer_info['lead_conversion_time'].head()

### Some statistics on the lead conversion times (measured in days) for the closed deals...  Notice that the standard deviation is higher than the mean value!  Let's see how closed deal revenue affects the lead conversion time.

In [0]:
cd_order_customer_info['lead_conversion_time'].describe()

In [0]:
cd_order_customer_info.sort_values(by ='lead_conversion_time',ascending=False).head(50)

In [0]:
revenue_per_mql = cd_order_customer_info.groupby(['mql_id','lead_conversion_time'])['total_payment'].sum().reset_index()
revenue_per_mql = revenue_per_mql[revenue_per_mql.total_payment != 0]  # Delete rows where sum('total_payment') = 0
cd_mqls_by_revenue = revenue_per_mql.sort_values(by='total_payment',ascending=False)
top_20_mqls= revenue_per_mql.sort_values(by='total_payment',ascending=False).head(20)

In [0]:

fig,ax = plt.subplots(figsize=(14,6))

x = top_20_mqls['mql_id']
y1 = top_20_mqls['lead_conversion_time']
y2 = top_20_mqls['total_payment']

plt.subplot(1,2,1)
plt.plot(x,y1, 'o-',color='red')
plt.title('Lead Conversion Time - Top 20 mql_ids by Revenue')
plt.ylabel('Lead Conversion Time (Days)')

plt.xticks(rotation=90)

ax2 = plt.subplot(1,2,2)
plt.bar(x, y2)
plt.title('Closed Deal Revenue')
plt.xlabel('mql_id')
plt.ylabel('Revenue')

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax2.yaxis.set_major_formatter(tick) 

plt.xticks(rotation=90)

plt.show()

In [0]:
fig, ax1 = plt.subplots(figsize=(14,6))

color = 'tab:blue'
ax1.set_ylabel('Revenue', color=color)  # we already handled the x-label with ax1
ax1.bar(top_20_mqls['mql_id'], top_20_mqls['total_payment'], color=color)
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax1.yaxis.set_major_formatter(tick) 
ax1.tick_params(axis='y', labelcolor=color)

plt.xticks(rotation=90)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_xlabel('mql_id')
ax2.set_ylabel('Lead Conversion Time (Days)', color=color, rotation=270,labelpad=15)
ax2.plot(top_20_mqls['mql_id'], top_20_mqls['lead_conversion_time'], color=color)
ax2.tick_params(axis='both', labelcolor=color)
ax2.grid(None)



fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

### What kind of customer review scores did the top closed deals generate? We have a sorted list of the closed deal 'mql_id's according to revenue generated in 'cd_mqls_by_revenue'. We need to generate a list of 'order_id's for each 'mql_id'.

In [0]:
mean_review_score_per_mql = cd_order_customer_info.groupby(['mql_id'])['review_score'].mean().reset_index()
mean_review_score_per_mql.rename(columns = {'review_score':'mean_review_score'},inplace=True)
mean_review_score_per_mql.dropna(inplace=True)

In [0]:
cd_mqls_by_revenue = cd_mqls_by_revenue.merge(mean_review_score_per_mql,how='left',on='mql_id')
top_mql_mean_scores = cd_mqls_by_revenue.head(20)

In [0]:
fig, ax1 = plt.subplots(figsize=(14,6))

color = 'tab:blue'
ax1.set_ylabel('Revenue', color=color)  # we already handled the x-label with ax1
ax1.bar(top_mql_mean_scores['mql_id'], top_mql_mean_scores['total_payment'], color=color)
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax1.yaxis.set_major_formatter(tick) 
ax1.tick_params(axis='y', labelcolor=color)

plt.xticks(rotation=90)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_xlabel('mql_id')
ax2.set_ylabel('Mean Review Score', color=color, rotation=270,labelpad=15)
ax2.plot(top_mql_mean_scores['mql_id'], top_mql_mean_scores['mean_review_score'], color=color)
ax2.tick_params(axis='both', labelcolor=color)
ax2.grid(None)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

### Let's look at some other characteristics of the top closed deal 'mql_id's...

We'll look at:

*  Number of 'order_id's
*  Number of 'customer_id's
*  Number of 'product_id's






In [0]:
dfs_to_merge = [cd_mqls_by_revenue,order_ids_per_mql,num_customers_per_cd_mql,num_products_per_cd_mql]
cd_mql_by_revenue = reduce(lambda left,right: pd.merge(left,right,on=['mql_id'],how='left'), dfs_to_merge)
cd_mql_by_revenue